# Extract modelling domain
The downloaded soil texture class map covers most of the globe. This script subsets the map to the domain of interest.

In [15]:
import os
from osgeo import gdal
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Control file handling

In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_control_files')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find where the data is

In [6]:
# Find the path where the raw soil maps need to go
soil_raw_path = read_from_control(controlFolder/controlFile,'parameter_soil_raw_path')

In [7]:
# Specify the default paths if required 
if soil_raw_path == 'default':
    soil_raw_path = make_default_path('parameters/soilclass/1_soil_classes_global') # outputs a Path()
else:
    soil_raw_path = Path(soil_raw_path) # make sure a user-specified path is a Path()

#### Find where the new map needs to go

In [8]:
# Find the path where the subset soil map need to go
soil_domain_path = read_from_control(controlFolder/controlFile,'parameter_soil_domain_path')
soil_domain_name = read_from_control(controlFolder/controlFile,'parameter_soil_tif_name')

In [9]:
# Specify the default paths if required 
if soil_domain_path == 'default':
    soil_domain_path = make_default_path('parameters/soilclass/2_soil_classes_domain') # outputs a Path()
else:
    soil_domain_path = Path(soil_domain_path) # make sure a user-specified path is a Path()

In [10]:
# Make the folder if it doesn't exist
soil_domain_path.mkdir(parents=True, exist_ok=True)

#### Find the extent of the domain

In [27]:
# Find which locations to download
coordinates = read_from_control(controlFolder/controlFile,'forcing_raw_space')

In [29]:
# Split coordinates into the format the download interface needs
coordinates = coordinates.split('/')

In [80]:
# Transcribe into bounding box
bbox = (float(coordinates[1]),float(coordinates[0]),float(coordinates[3]),float(coordinates[2]))

#### Open the map and do the subsetting
Source: https://gis.stackexchange.com/questions/244667/cropping-area-of-interest-from-raster-file-using-python

In [20]:
# Find the name of the source file
for file in os.listdir(soil_raw_path):
    if file.endswith('.tif'):
        soil_raw_name = file

In [54]:
# Convert paths to strings
old_tif = str(soil_raw_path/soil_raw_name)
new_tif = str(soil_domain_path/soil_domain_name)

In [86]:
# Crop to domain
ds = gdal.Translate(new_tif, old_tif, projWin=bbox)

In [87]:
# Close the data set
ds = None

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [90]:
# Set the log path and file name
logPath = soil_domain_path
log_suffix = '_soilclass_cropping_log.txt'

In [91]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [92]:
# Copy this script
thisFile = 'extract_domain.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [93]:
# Get current date and time
now = datetime.now()

In [94]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Cropped SOILGRIDS-derived soil texture class map to local domain.']
    for txt in lines:
        file.write(txt)  